# Transcript Utilities
This notebook provides functions for transcribing audio using OpenAI Whisper, with optional SRT/VTT output.

In [ ]:
# Install required packages (uncomment if needed)
# %pip install openai-whisper

In [ ]:
import sys
import argparse

# Try to import Whisper for speech-to-text
try:
    import whisper
except ImportError:
    print("The 'whisper' package is not installed. Please see the instructions below.")
    sys.exit(1)

# Optional: For SRT/VTT subtitle output
try:
    from whisper.utils import write_srt, write_vtt
except ImportError:
    write_srt = None
    write_vtt = None

def transcribe_audio(
    input_audio,
    output_txt=None,
    output_srt=None,
    output_vtt=None,
    model_size='large',
    language=None
):
    """
    Transcribe audio using OpenAI Whisper and save results in text, SRT, and VTT formats.
    Args:
        input_audio (str): Path to the input audio file (any format supported by Whisper)
        output_txt (str): Path to save the plain text transcript
        output_srt (str, optional): Path to save SRT subtitles
        output_vtt (str, optional): Path to save VTT subtitles
        model_size (str): Whisper model size (tiny, base, small, medium, large)
        language (str, optional): Language code (e.g., 'en') or None for auto-detect
    """
    model = whisper.load_model(model_size)
    print(f"Loaded Whisper model: {model_size}")
    try:
        result = model.transcribe(input_audio, language=language, verbose=True, task='transcribe')
    except Exception as e:
        print("Audio loading failed. Make sure your input file is a 16-bit, 16kHz, mono WAV file.")
        print(f"Error: {e}")
        raise
    transcript = result['text'].strip()

    # Write plain text output if output_txt is provided
    if output_txt:
        with open(output_txt, 'w', encoding='utf-8') as f:
            f.write(transcript + '\n')
        print(f"Transcription saved to {output_txt}")

    # Write SRT subtitles if requested and supported
    if output_srt and write_srt:
        with open(output_srt, 'w', encoding='utf-8') as f:
            write_srt(result['segments'], file=f)
        print(f"SRT subtitles saved to {output_srt}")
    # Write VTT subtitles if requested and supported
    if output_vtt and write_vtt:
        with open(output_vtt, 'w', encoding='utf-8') as f:
            write_vtt(result['segments'], file=f)
        print(f"VTT subtitles saved to {output_vtt}")

    # Print detected language
    print(f"Detected language: {result['language']}")
    return transcript